In [ ]:
!pip install fuzzywuzzy
!pip install python-Levenshtein

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 19.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import re
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

Labels : product_id, product_name, product_fullName, product_url, text, cleaned_text, review_date, category


# Tech Reviews

In [ ]:
df_tech = pd.read_csv('./tech_reviews_v3.csv')
df_tech

,product_id,product_name,product_fullName,product_url,text,cleaned_text,review_date,category
0,1333512,11th-Gen-IntelR-CoreTM-i3-1115G4---300GHz,11th-Gen-IntelR-CoreTM-i3-1115G4-300GHz,https://cpu.userbenchmark.com/SpeedTest/133351...,nan works perfectly fine for school and work,"['nan', 'works', 'perfectly', 'fine', 'school'...",15 days ago.,1
1,1333512,11th-Gen-IntelR-CoreTM-i3-1115G4---300GHz,11th-Gen-IntelR-CoreTM-i3-1115G4-300GHz,https://cpu.userbenchmark.com/SpeedTest/133351...,Poor Keeps spiking to 100% and giving major pe...,"['poor', 'keeps', 'spiking', 'giving', 'major'...",7 months ago.,0
2,1333512,11th-Gen-IntelR-CoreTM-i3-1115G4---300GHz,11th-Gen-IntelR-CoreTM-i3-1115G4-300GHz,https://cpu.userbenchmark.com/SpeedTest/133351...,"Excellent great processor for basic functions,...","['excellent', 'great', 'processor', 'basic', '...",4 months ago.,1
3,1333512,11th-Gen-IntelR-CoreTM-i3-1115G4---300GHz,11th-Gen-IntelR-CoreTM-i3-1115G4-300GHz,https://cpu.userbenchmark.com/SpeedTest/133351...,Average 2C 4T mobile CPU. Base clock of 1.7Ghz...,"['average', '2c', '4t', 'mobile', 'cpu', 'base...",3 years ago.,1
4,1418235,11th-Gen-IntelR-CoreTM-i5-1130G7---110GHz,11th-Gen-IntelR-CoreTM-i5-1130G7-110GHz,https://cpu.userbenchmark.com/SpeedTest/141823...,Excellent 11th Gen i5 with 4C 8T. Base clock o...,"['excellent', '11th', 'gen', 'i5', '4c', '8t',...",2 years ago.,1
...,...,...,...,...,...,...,...,...
4433,451,PentiumR-Dual-Core-CPU-------T4200----200GHz,PentiumR-Dual-Core-CPU-T4200-200GHz,https://cpu.userbenchmark.com/SpeedTest/451/Pe...,Excellent 10 years old but still enough for br...,"['excellent', 'years', 'old', 'still', 'enough...",4 years ago.,1
4434,451,PentiumR-Dual-Core-CPU-------T4200----200GHz,PentiumR-Dual-Core-CPU-T4200-200GHz,https://cpu.userbenchmark.com/SpeedTest/451/Pe...,Excellent It is 11 years old and works perfect...,"['excellent', 'years', 'old', 'works', 'perfec...",4 years ago.,1
4435,2938,PentiumR-Dual-Core-CPU-------T4300----210GHz,PentiumR-Dual-Core-CPU-T4300-210GHz,https://cpu.userbenchmark.com/SpeedTest/2938/P...,Good My first notebook Acer Gateway NV44 have ...,"['good', 'first', 'notebook', 'acer', 'gateway...",4 years ago.,1
4436,2938,PentiumR-Dual-Core-CPU-------T4300----210GHz,PentiumR-Dual-Core-CPU-T4300-210GHz,https://cpu.userbenchmark.com/SpeedTest/2938/P...,Good My first notebook Acer Gateway NV44 have ...,"['good', 'first', 'notebook', 'acer', 'gateway...",4 years ago.,1


Outcome : Tech Reviews are correctly displayed. -> Need to add product_name : which shall have only the intel processor name

# User Reviews

In [ ]:
df_user = pd.read_csv('./user_reviews_v3.csv')
df_user

,product_id,product_name,product_fullName,product_url,text,cleaned_text,review_date,category
0,itmfa5j4jcma4px6,I7,intel-i7-3770-3-9-ghz-lga-1155-socket-4-cores-...,https://www.flipkart.com/intel-i7-3770-3-9-ghz...,Delightful Seems to be performing as expected....,"['delightful', 'seems', 'performing', 'expecte...","Aug, 2022",1
1,itmfa5j4jcma4px6,I7,intel-i7-3770-3-9-ghz-lga-1155-socket-4-cores-...,https://www.flipkart.com/intel-i7-3770-3-9-ghz...,"Fabulous! Okay so, at first i was worried as h...","['fabulous', 'okay', 'first', 'worried', 'hell...","Sep, 2022",1
2,itmfa5j4jcma4px6,I7,intel-i7-3770-3-9-ghz-lga-1155-socket-4-cores-...,https://www.flipkart.com/intel-i7-3770-3-9-ghz...,Terrific Pacaging was not good and obviously i...,"['terrific', 'pacaging', 'good', 'obviously', ...","Oct, 2022",1
3,itmfa5j4jcma4px6,I7,intel-i7-3770-3-9-ghz-lga-1155-socket-4-cores-...,https://www.flipkart.com/intel-i7-3770-3-9-ghz...,"Terrific purchase Processor is very good, its ...","['terrific', 'purchase', 'processor', 'good', ...","Aug, 2022",1
4,itmfa5j4jcma4px6,I7,intel-i7-3770-3-9-ghz-lga-1155-socket-4-cores-...,https://www.flipkart.com/intel-i7-3770-3-9-ghz...,Brilliant Good mast candishen power full proce...,"['brilliant', 'good', 'mast', 'candishen', 'po...","May, 2023",1
...,...,...,...,...,...,...,...,...
1175,itm0ffcf9089ca76,I3,intel-i3-12100f-4-3-ghz-upto-lga1700-socket-4-...,https://www.flipkart.com/intel-i3-12100f-4-3-g...,Perfect product! Good ???,"['perfect', 'product', 'good']","Feb, 2023",1
1176,itm0ffcf9089ca76,I3,intel-i3-12100f-4-3-ghz-upto-lga1700-socket-4-...,https://www.flipkart.com/intel-i3-12100f-4-3-g...,Excellent Supb. Product,"['excellent', 'supb', 'product']","Jan, 2023",1
1177,itm3a4e911ab1b55,I9,intel-i9-13900kf-2-2-ghz-lga1700-socket-8-core...,https://www.flipkart.com/intel-i9-13900kf-2-2-...,Excellent Get performance for machine learning...,"['excellent', 'get', 'performance', 'machine',...",11 months ago,1
1178,itm1c0df8b3313fa,Unknown,intel-core-i7-2600-2nd-gen-quad-core-8mb-cache...,https://www.flipkart.com/intel-core-i7-2600-2n...,Useless product Not working and waste of money...,"['useless', 'product', 'working', 'waste', 'mo...","Apr, 2023",0


OutCome: We have successfully got correct labels and category. Product Name is still an issue

# Product Names Search by Text

In [ ]:
df_user = pd.read_csv('./user_reviews_v4.csv')
df_tech = pd.read_csv('./tech_reviews_v4.csv')

In [ ]:
def fuzzy_search(row, target_text):
    match_score = fuzz.partial_ratio(target_text, row['product_name'])
    return match_score >= 90

def filter_dataframe(df, target_text):
    return df[df.apply(fuzzy_search, axis=1, target_text=target_text)]

target_text = 'intel-i7-4790'
filtered_df = filter_dataframe(df_user, target_text)
filtered_df

,product_id,product_name,product_fullName,product_url,text,cleaned_text,review_date,category
44,itmdyzqrz3bgz2du,intel-i7-4790,intel-i7-4790-4-ghz-lga-1150-socket-cores-desk...,https://www.flipkart.com/intel-i7-4790-4-ghz-l...,excessive power I am really impressed by Intel...,"['excessive', 'power', 'really', 'impressed', ...","Sep, 2014",1
45,itmdyzqrz3bgz2du,intel-i7-4790,intel-i7-4790-4-ghz-lga-1150-socket-cores-desk...,https://www.flipkart.com/intel-i7-4790-4-ghz-l...,good one but not the best its a good processor...,"['good', 'one', 'best', 'good', 'processor', '...","Oct, 2014",1
46,itmdyzqrz3bgz2du,intel-i7-4790,intel-i7-4790-4-ghz-lga-1150-socket-cores-desk...,https://www.flipkart.com/intel-i7-4790-4-ghz-l...,Value-for-money Good performance high speed work.,"['value-for-money', 'good', 'performance', 'hi...",4 months ago,1
47,itmdyzqrz3bgz2du,intel-i7-4790,intel-i7-4790-4-ghz-lga-1150-socket-cores-desk...,https://www.flipkart.com/intel-i7-4790-4-ghz-l...,Value-for-money Now working,"['value-for-money', 'working']",6 months ago,1
48,itmdyzqrz3bgz2du,intel-i7-4790,intel-i7-4790-4-ghz-lga-1150-socket-cores-desk...,https://www.flipkart.com/intel-i7-4790-4-ghz-l...,Worth every penny I like it,"['worth', 'every', 'penny', 'like']",7 months ago,1
49,itmdyzqrz3bgz2du,intel-i7-4790,intel-i7-4790-4-ghz-lga-1150-socket-cores-desk...,https://www.flipkart.com/intel-i7-4790-4-ghz-l...,Great product Nice package ?? nd working fine ...,"['great', 'product', 'nice', 'package', 'nd', ...",2 months ago,1
50,itmdyzqrz3bgz2du,intel-i7-4790,intel-i7-4790-4-ghz-lga-1150-socket-cores-desk...,https://www.flipkart.com/intel-i7-4790-4-ghz-l...,Perfect product! The product is great!!! The p...,"['perfect', 'product', 'product', 'great', 'pa...",5 months ago,1
51,itmdyzqrz3bgz2du,intel-i7-4790,intel-i7-4790-4-ghz-lga-1150-socket-cores-desk...,https://www.flipkart.com/intel-i7-4790-4-ghz-l...,Must buy! High Parfomence Processor product pa...,"['must', 'buy', 'high', 'parfomence', 'process...",9 days ago,1
52,itmdyzqrz3bgz2du,intel-i7-4790,intel-i7-4790-4-ghz-lga-1150-socket-cores-desk...,https://www.flipkart.com/intel-i7-4790-4-ghz-l...,Unsatisfactory Do not buy a 4th generation pro...,"['unsatisfactory', 'buy', '4th', 'generation',...",17 days ago,0
53,itmdyzqrz3bgz2du,intel-i7-4790,intel-i7-4790-4-ghz-lga-1150-socket-cores-desk...,https://www.flipkart.com/intel-i7-4790-4-ghz-l...,Excellent Good,"['excellent', 'good']",2 months ago,1


# Helper Functions

In [ ]:
def extract_product_details_flipkart(url):
    pattern = r"https://www\.flipkart\.com/(?P<product_fullname>[^/]+)/product-reviews/(?P<product_id>[^?]+)"
    product_name_regex = r"intel-(?P<product_name>\w+\d+)"

    match = re.match(pattern, url)
    if match:
        product_id = match.group("product_id")
        product_fullname = match.group("product_fullname")

        name_match = re.search(product_name_regex, product_fullname)
        if name_match:
            product_name = name_match.group("product_name").upper()
        else:
            product_name = "Unknown"

        return product_id, product_fullname, product_name

    return None, None, None

In [ ]:
def extract_product_details_userbench(url):
    pattern1 = r'https://cpu\.userbenchmark\.com/SpeedTest/(\d+)/(.*)'
    pattern2 = r'https://cpu\.userbenchmark\.com/(.*)/Rating/(\d+)'

    match1 = re.match(pattern1, url)
    if match1:
        product_id = match1.group(1)
        product_fullname = f"{match1.group(1)} {match1.group(2)}"
        product_name = match1.group(2)
        return product_id, product_fullname, product_name

    match2 = re.match(pattern2, url)
    if match2:
        product_id = match2.group(2)
        product_fullname = f"{match2.group(1)} {match2.group(2)}"
        product_name = match2.group(1)
        return product_id, product_fullname, product_name

    return None, None, None

In [ ]:
def fuzzy_search(row, target_text):
    match_score = fuzz.partial_ratio(target_text, row['product_fullName'])
    return match_score >= 90

def filter_dataframe(df, target_text):
    return df[df.apply(fuzzy_search, axis=1, target_text=target_text)]

# Fixing Product Names of Intel CPUs

In [ ]:
df_user = pd.read_csv('./user_reviews_v3.csv')
df_user

,product_id,product_name,product_fullName,product_url,text,cleaned_text,review_date,category
0,itmfa5j4jcma4px6,I7,intel-i7-3770-3-9-ghz-lga-1155-socket-4-cores-...,https://www.flipkart.com/intel-i7-3770-3-9-ghz...,Delightful Seems to be performing as expected....,"['delightful', 'seems', 'performing', 'expecte...","Aug, 2022",1
1,itmfa5j4jcma4px6,I7,intel-i7-3770-3-9-ghz-lga-1155-socket-4-cores-...,https://www.flipkart.com/intel-i7-3770-3-9-ghz...,"Fabulous! Okay so, at first i was worried as h...","['fabulous', 'okay', 'first', 'worried', 'hell...","Sep, 2022",1
2,itmfa5j4jcma4px6,I7,intel-i7-3770-3-9-ghz-lga-1155-socket-4-cores-...,https://www.flipkart.com/intel-i7-3770-3-9-ghz...,Terrific Pacaging was not good and obviously i...,"['terrific', 'pacaging', 'good', 'obviously', ...","Oct, 2022",1
3,itmfa5j4jcma4px6,I7,intel-i7-3770-3-9-ghz-lga-1155-socket-4-cores-...,https://www.flipkart.com/intel-i7-3770-3-9-ghz...,"Terrific purchase Processor is very good, its ...","['terrific', 'purchase', 'processor', 'good', ...","Aug, 2022",1
4,itmfa5j4jcma4px6,I7,intel-i7-3770-3-9-ghz-lga-1155-socket-4-cores-...,https://www.flipkart.com/intel-i7-3770-3-9-ghz...,Brilliant Good mast candishen power full proce...,"['brilliant', 'good', 'mast', 'candishen', 'po...","May, 2023",1
...,...,...,...,...,...,...,...,...
1175,itm0ffcf9089ca76,I3,intel-i3-12100f-4-3-ghz-upto-lga1700-socket-4-...,https://www.flipkart.com/intel-i3-12100f-4-3-g...,Perfect product! Good ???,"['perfect', 'product', 'good']","Feb, 2023",1
1176,itm0ffcf9089ca76,I3,intel-i3-12100f-4-3-ghz-upto-lga1700-socket-4-...,https://www.flipkart.com/intel-i3-12100f-4-3-g...,Excellent Supb. Product,"['excellent', 'supb', 'product']","Jan, 2023",1
1177,itm3a4e911ab1b55,I9,intel-i9-13900kf-2-2-ghz-lga1700-socket-8-core...,https://www.flipkart.com/intel-i9-13900kf-2-2-...,Excellent Get performance for machine learning...,"['excellent', 'get', 'performance', 'machine',...",11 months ago,1
1178,itm1c0df8b3313fa,Unknown,intel-core-i7-2600-2nd-gen-quad-core-8mb-cache...,https://www.flipkart.com/intel-core-i7-2600-2n...,Useless product Not working and waste of money...,"['useless', 'product', 'working', 'waste', 'mo...","Apr, 2023",0


In [ ]:
df_tech = pd.read_csv('./tech_reviews_v3.csv')
df_tech

,product_id,product_url,product_name,product_fullName,review_date,text,cleaned_text,category
0,1333512,https://cpu.userbenchmark.com/SpeedTest/133351...,11th-Gen-IntelR-CoreTM-i3-1115G4---300GHz,11th-Gen-IntelR-CoreTM-i3-1115G4-300GHz,15 days ago.,nan works perfectly fine for school and work,"['nan', 'works', 'perfectly', 'fine', 'school'...",1
1,1333512,https://cpu.userbenchmark.com/SpeedTest/133351...,11th-Gen-IntelR-CoreTM-i3-1115G4---300GHz,11th-Gen-IntelR-CoreTM-i3-1115G4-300GHz,7 months ago.,Poor Keeps spiking to 100% and giving major pe...,"['poor', 'keeps', 'spiking', 'giving', 'major'...",0
2,1333512,https://cpu.userbenchmark.com/SpeedTest/133351...,11th-Gen-IntelR-CoreTM-i3-1115G4---300GHz,11th-Gen-IntelR-CoreTM-i3-1115G4-300GHz,4 months ago.,"Excellent great processor for basic functions,...","['excellent', 'great', 'processor', 'basic', '...",1
3,1333512,https://cpu.userbenchmark.com/SpeedTest/133351...,11th-Gen-IntelR-CoreTM-i3-1115G4---300GHz,11th-Gen-IntelR-CoreTM-i3-1115G4-300GHz,3 years ago.,Average 2C 4T mobile CPU. Base clock of 1.7Ghz...,"['average', '2c', '4t', 'mobile', 'cpu', 'base...",1
4,1418235,https://cpu.userbenchmark.com/SpeedTest/141823...,11th-Gen-IntelR-CoreTM-i5-1130G7---110GHz,11th-Gen-IntelR-CoreTM-i5-1130G7-110GHz,2 years ago.,Excellent 11th Gen i5 with 4C 8T. Base clock o...,"['excellent', '11th', 'gen', 'i5', '4c', '8t',...",1
...,...,...,...,...,...,...,...,...
4433,451,https://cpu.userbenchmark.com/SpeedTest/451/Pe...,PentiumR-Dual-Core-CPU-------T4200----200GHz,PentiumR-Dual-Core-CPU-T4200-200GHz,4 years ago.,Excellent 10 years old but still enough for br...,"['excellent', 'years', 'old', 'still', 'enough...",1
4434,451,https://cpu.userbenchmark.com/SpeedTest/451/Pe...,PentiumR-Dual-Core-CPU-------T4200----200GHz,PentiumR-Dual-Core-CPU-T4200-200GHz,4 years ago.,Excellent It is 11 years old and works perfect...,"['excellent', 'years', 'old', 'works', 'perfec...",1
4435,2938,https://cpu.userbenchmark.com/SpeedTest/2938/P...,PentiumR-Dual-Core-CPU-------T4300----210GHz,PentiumR-Dual-Core-CPU-T4300-210GHz,4 years ago.,Good My first notebook Acer Gateway NV44 have ...,"['good', 'first', 'notebook', 'acer', 'gateway...",1
4436,2938,https://cpu.userbenchmark.com/SpeedTest/2938/P...,PentiumR-Dual-Core-CPU-------T4300----210GHz,PentiumR-Dual-Core-CPU-T4300-210GHz,4 years ago.,Good My first notebook Acer Gateway NV44 have ...,"['good', 'first', 'notebook', 'acer', 'gateway...",1


In [ ]:
fullname_user  = df_user['product_fullName'].unique()
fullname_tech = df_tech['product_fullName'].unique()

print(fullname_user.shape, fullname_tech.shape)

(72,) (744,)


In [ ]:
result = pd.concat([pd.Series(fullname_user), pd.Series(fullname_tech)], ignore_index=True)
result.columns = ['new_col']
result

0      intel-i7-3770-3-9-ghz-lga-1155-socket-4-cores-...
1      intel-i7-4790-4-ghz-lga-1150-socket-cores-desk...
2      intel-3570k-3-4-ghz-lga-1155-socket-4-cores-de...
3      intel-i5-3570-3-4-ghz-upto-4-lga-1155-socket-c...
4      intel-i7-13700k-2-5-ghz-lga1700-socket-8-cores...
                             ...                        
811                  PentiumR-Dual-Core-CPU-E6500-293GHz
812                  PentiumR-Dual-Core-CPU-E6600-306GHz
813                  PentiumR-Dual-Core-CPU-T4200-200GHz
814                  PentiumR-Dual-Core-CPU-T4300-210GHz
815                  PentiumR-Dual-Core-CPU-T4500-230GHz
Length: 816, dtype: object

In [ ]:
result.to_csv('product_names.csv', index=False)

In [ ]:
!pip install selenium
!apt update
!apt install chromium-chromedriver

In [ ]:
# Necessary Imports

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
import os
import csv
from random import uniform
import re

In [ ]:
df_name = pd.read_csv('product_names.csv')

In [ ]:
# Chromium Headless Options

chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--remote-debugging-port=9222")

driver = webdriver.Chrome(options=chrome_options)
pattern = (r"https://www\.intel\.com/content/www/us/en/products/sku/.+/specifications\.html")


names_dict = {"product_name": [], "product_fullName": []}

with open("product_names.csv", mode="r") as file:
    csv_reader = csv.DictReader(file)
    for row in csv_reader:
        baseUrl = row["URL"]
        google_search = "https://www.google.com/search?q=" + baseUrl

        driver.get(google_search)
        content_div = driver.find_element(By.ID, "rcnt").get_attribute("outerHTML")
        page_html = BeautifulSoup(content_div, "html.parser")
        a_tags = page_html.find_all("a")

        for tag in a_tags:
            url = tag.get("href")
            if(url is None):
                continue
            match = re.match(pattern, url)
            if match:
                driver.get(url)
                processor_html = driver.find_element(
                    By.ID, "specs-1-0-0"
                ).get_attribute("outerHTML")
                processor_bs4 = BeautifulSoup(processor_html, "html.parser")
                a = processor_bs4.find_all(
                    "div", class_="col-xs-6 col-lg-6 tech-data"
                )
                product_name = a[3].get_text().strip()

                print(product_name)

                names_dict["product_name"].append(product_name)
                names_dict["product_fullName"].append(baseUrl)

        driver.implicitly_wait(5)


df = pd.DataFrame(names_dict)
while True:
    if os.path.isfile(f"./product_names/{baseUrl}.csv"):
        baseUrl = baseUrl + "-" + str(int(uniform(1, 100)))
    else:
        break
csv_filename = f"./product_names/{baseUrl}.csv"
df.to_csv(csv_filename, index=False)
print(f"Data saved to {csv_filename}")
